THIS IS A ENG-SPN VOICE TRANSLATOR. PLEASE RUN ALL THE CELLS AND FOLLOW THE VOICE-OVER INSTRUCTIONS IN THE LAST CELL FOR SMOOTH ESCALATTION.

In [3]:
!pip install SpeechRecognition deep-translator gTTS pydub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 56.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 8.9 MB/s eta 0:00:00
  Attempting uninstall: click
    Found existing installation: click 8.2.1
    Uninstalling click-8.2.1:
      Successfully uninstalled click-8.2.1


In [2]:
!pip install gTTS

##  Microphone and Audio Playback Functions

In [4]:
from IPython.display import Javascript, display, Audio # Imports a special set of Colab functions to display things like JavaScript and audio.
from base64 import b64decode # Used to decode the base64-encoded audio data from the browser.
from google.colab import output # Used to interact with the Colab notebook's output.
from gtts import gTTS # Imports the gTTS library for converting text to speech.
import io # Imports the io module for handling in-memory files.
import speech_recognition as sr # Imports the SpeechRecognition library for converting speech to text.
from deep_translator import GoogleTranslator # Imports the GoogleTranslator from deep_translator to handle text translation.
from pydub import AudioSegment # Imports pydub to handle audio file manipulations.
import time # Imports the time module for pauses and delays.
import tempfile # Imports tempfile to create temporary files for audio storage.
import sys # Imports sys module for system-specific parameters and functions.
import threading # Imports threading to handle multiple threads if needed.

In [5]:
# JavaScript to create record button in the notebook

RECORD= """
const sleep= time => new Promise(resolve => setTimeout(resolve, time))   // Defines a JavaScript function to pause execution for a specific time.
const b2text= blob => new Promise(resolve => {    // Defines a function to convert a binary audio blob into a base64-encoded string.
  const reader = new FileReader()                 // Creates a new FileReader object.
  reader.onloadend = () => resolve(reader.result) // Sets up an event listener for the onloadend event of the reader.
  reader.readAsDataURL(blob)                      // Reads the audio blob as a data URL.
})
var record = time => new Promise(async resolve => {                  // Main recording function that takes a duration.
  stream= await navigator.mediaDevices.getUserMedia({ audio: true }) // Requests access to the user's microphone.
  recorder= new MediaRecorder(stream)                                // Creates a new media recorder.
  chunks= []                                                         // Initializes an array to store audio data chunks.
  recorder.ondataavailable = e => chunks.push(e.data)                // Adds incoming audio data to the chunks array.
  recorder.start()                                                   // Starts the recording.
  await sleep(time)                                                  // Pauses for the specified duration using the sleep function.
  recorder.onstop = async ()=>{                                      // Defines what to do when the recording stops.
    blob= new Blob(chunks)                                           // Combines the chunks into a single audio blob.
    text= await b2text(blob)                                         // Converts the blob into a base64-encoded string.
    resolve(text)                                                    // Resolves the promise with the audio data.
  }
  recorder.stop()                                                    // Stops the recording after the timer ends.
})
"""

In [22]:
import os

In [10]:
def record_audio(filename="audio.wav", duration=5):
  """
  Records audio from the browser's microphone for a specified duration and saves it as a WAV file.
  Args:
        filename (str): The name of the file to save the audio.
        duration (int): The duration of the recording in seconds.
    Returns:
        str: The filename of the recorded audio.
  """
  input(f"Recording for {duration} seconds... Please speak now.") # Prints a message to the user.
  display(Javascript(RECORD)) # Executes the JavaScript code to show the record button.
  s = output.eval_js('record(%d)' % (duration * 1000)) # Calls the JavaScript function and gets the audio data as a string.
  b = b64decode(s.split(',')[1]) # Decodes the base64 string back into binary audio data.
  with open(filename, 'wb') as f: # Opens a new file in write-binary mode.
    f.write(b) # Writes the binary audio data to the file.
  print("Recording finished.") # Informs the user that the recording is complete.
  return filename # Returns the name of the file where the audio is saved.

In [23]:
def speak_colab(text, lang="en"):
  """
  Converts text to speech and plays the audio in the notebook output.
  Args:
        text (str): The text to be spoken.
        lang (str): The language code for the text.
  """
  try:
    tts= gTTS(text=text, lang=lang) # Creates a gTTS object with the specified text and language.
    with tempfile.NamedTemporaryFile(suffix=".mp3", delete=False) as fp: # Creates a temporary MP3 file to store the audio.
        tts.save(fp.name) # Saves the gTTS audio to the temporary file.
        audio_data= open(fp.name, "rb").read() # Reads the audio data from the file.
        display(Audio(audio_data, autoplay=True)) # Displays the audio in the notebook.
        os.remove(fp.name)                        # Deletes the temporary file after playing.

  except Exception as e:
    print(f"Error during Text-to-Speech:{e}")    # Prints an error message if something goes wrong.

## THE MAIN TRANSLATION LOGIC

This cell contains the functions to recognize and translate the audio.

In [18]:
def recognize_and_translate(audio_file_path, source_lang, dest_lang):
  """
  Recognizes speech in an audio file, translates it and prints the result.
  Args:
        audio_file_path (str): The path to the audio file.
        source_lang (str): The language of the audio to be recognized.
        dest_lang (str): The language to translate to.
  """
  r= sr.Recognizer()      # Initializes the speech recognition object.
  try:
      # Step 1: Explicitly load the audio file as a WebM file, as that is the format recorded by the browser.
      audio_segment = AudioSegment.from_file(audio_file_path, format="webm")

      # Create a temporary WAV file with a specific codec to ensure compatibility.
      with tempfile.NamedTemporaryFile(suffix=".wav", delete=False) as fp:
            audio_segment.export(fp.name, format="wav", codec="pcm_s16le")
            converted_audio_path = fp.name

      # Step 2: Use the valid WAV file for speech recognition.
      with sr.AudioFile(converted_audio_path) as source:  # Opens the audio file.
          audio= r.record(source) # Records the entire audio from the file.

      # Recognize speech using Google's service
      recognized_text= r.recognize_google(audio, language= source_lang) # Converts the audio to text using Google's API.
      print(f"Recognized ({source_lang} Text): {recognized_text}")      # Prints the recognized text.

      # Translate the text using deep-translator
      translator= GoogleTranslator(source= source_lang[:2], target= dest_lang[:2]) # Creates/Initializes the translator with source and target languages.
      translation= translator.translate(recognized_text)

      print(f"Translated ({dest_lang}): {translation}")  # Print the translated text.

      # Speak the translated text
      speak_colab(translation, dest_lang)     # Calls the function to play the translated text aloud.

  except sr.UnknownValueError:
      print("Google Speech Recognition could not understand audio")
  except sr.RequestError as e:
      print(f"Could not request results from Google Speech Recognition Service; {e}")
  except Exception as e:
        print(f"An unexpected error occurred during processing: {e}")

# The Main Loop

Final Program Creation.

In [26]:
def main():
  print("Starting real-time translation program...")  # Prints a starting process.
  print("This will run a two-way conversation loop.")  # Explains the flow of the program.

  # Speaker 1 (English to Spanish)
  print("\n--- Speaker 1's turn [English to Spanish] --->") # Indicates whose turn it is.
  print("Press ENTER key in the input block when you're ready to speak.")
  audio_file_en= record_audio(filename="speaker1.wav", duration=5) # Records 5 seconds of audio from the first speaker.
  recognize_and_translate(audio_file_en, 'en-US', 'es') # Recognizes and translates the English audio to Spanish.

  # Wait a moment for the audio to finish
  time.sleep(2) # Pauses the program to allow the translated audio to finish playing.

  # Speaker 2: Spanish to English
  print("\n--- Speaker 2's turn (Spanish -> English) ---") # Indicates the second speaker's turn.
  print("Press ENTER key in the input block when you're ready to speak.")
  audio_file_es = record_audio(filename="speaker2.wav", duration=5) # Records 5 seconds of audio from the second speaker.
  recognize_and_translate(audio_file_es, 'es-ES', 'en') # Recognizes and translates the Spanish audio to English.


if __name__ == "__main__":
    main() # Calls the main function to start the program.

Starting real-time translation program...
This will run a two-way conversation loop.

--- Speaker 1's turn [English to Spanish] --->
Press ENTER key in the input block when you're ready to speak.
Recording for 5 seconds... Please speak now.


<IPython.core.display.Javascript object>

Recording finished.
Recognized (en-US Text): hello
Translated (es): Hola



--- Speaker 2's turn (Spanish -> English) ---
Press ENTER key in the input block when you're ready to speak.
Recording for 5 seconds... Please speak now.


<IPython.core.display.Javascript object>

Recording finished.
Recognized (es-ES Text): hola
Translated (en): hello
